In [1]:
%cd ../
%pwd

/home/rr/Documents/GitProj/aa_on_vad


/home/rr/anaconda3/envs/aaml/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


'/home/rr/Documents/GitProj/aa_on_vad'

In [2]:
import soundfile as sf
from pathlib import Path
import numpy as np
import torch
import IPython
from functools import partial
play = partial(IPython.display.Audio,
               rate=16000)
playr = IPython.display.Audio

In [46]:
import plotly.graph_objects as go


def plot(y:list, names:list=None, X=None):
    trace = []
    colors = [
        'Blue',
        'Orange',
        'Green',
        'Red',
        'Purple',
        'Magenta',
        'Cyan',
        'Brown',
        'Pink',
        'Lime',
        'Yellow',
        'Teal',
        'Olive',
        'Navy',
        'Maroon',
        'Coral',
        'Gold',
        'Indigo',
        'Turquoise',
        'Lavender',
        'Mint',
        'Silver',
        ]
    if names:
        for ik, y_ in enumerate(y):
            trace.append(go.Scatter(x=np.arange(len(y_)) if X==None else X, y=y_, mode='lines', name=names[ik], line=dict(color=colors[ik]), opacity=0.5))
    else:
        for ik, y_ in enumerate(y):
            trace.append(go.Scatter(x=np.arange(len(y_)) if X==None else X, y=y_, mode='lines', name=f'arg # {ik + 1}', line=dict(color=colors[ik]), opacity=0.5))
    
    # Combining both traces into one figure
    fig = go.Figure(data=trace,)

    # Setting the layout
    fig.update_layout(
        title='Two Line Charts on One Plot',
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        showlegend=True
    )

    # Display the plot
    fig.show()

In [4]:
from src.datasets.mixed_vad_datset import get_datset
from src.models.adversarial_models.CNN1d_based import NoiseGenerator
from src.models.silero.evaluate_model import validate_silero_vad, print_metrics
import src.models.silero.train as tl
from src.models.silero.loss import AdversarialLoss

In [5]:
print('Loading dataset...')
train_ds = get_datset(mode='train', erase_silence=True)
eval_ds = get_datset(mode='eval', erase_silence=True)
print('Dataset loaded!')

Loading dataset...
Dataset loaded!


In [5]:
model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad')
print(model)

/home/rr/anaconda3/envs/aaml/lib/python3.11/site-packages/torch/hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /home/rr/.cache/torch/hub/master.zip


RecursiveScriptModule(
  original_name=VADRNNJITMerge
  (_model): RecursiveScriptModule(
    original_name=VADRNNJIT
    (stft): RecursiveScriptModule(
      original_name=STFT
      (padding): RecursiveScriptModule(original_name=ReflectionPad1d)
    )
    (encoder): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=SileroVadBlock
        (se): RecursiveScriptModule(original_name=Identity)
        (activation): RecursiveScriptModule(original_name=ReLU)
        (reparam_conv): RecursiveScriptModule(original_name=Conv1d)
      )
      (1): RecursiveScriptModule(
        original_name=SileroVadBlock
        (se): RecursiveScriptModule(original_name=Identity)
        (activation): RecursiveScriptModule(original_name=ReLU)
        (reparam_conv): RecursiveScriptModule(original_name=Conv1d)
      )
      (2): RecursiveScriptModule(
        original_name=SileroVadBlock
        (se): RecursiveScriptModule(original_name=Identity)
     

In [7]:
print(f'Noise producer model has {sum(p.numel() for p in model.parameters())} parametes')


Noise producer model has 462594 parametes


In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [10]:
amodel = NoiseGenerator()

In [11]:
print(amodel) 

NoiseGenerator(
  (encoder): ModuleList(
    (0): Sequential(
      (0): Conv1d(1, 32, kernel_size=(1601,), stride=(2,), padding=(800,))
      (1): InstanceNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (1): Sequential(
      (0): Conv1d(32, 64, kernel_size=(801,), stride=(2,), padding=(400,))
      (1): InstanceNorm1d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (2): Sequential(
      (0): Conv1d(64, 128, kernel_size=(401,), stride=(2,), padding=(200,))
      (1): InstanceNorm1d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv1d(128, 256, kernel_size=(201,), stride=(2,), padding=(100,))
      (1): InstanceNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): LeakyReLU(negative_slope=0.2)
    

In [12]:
metrics = validate_silero_vad(
        model,
        eval_ds,
        1,
        # noise_producer,
    )
print_metrics(metrics)

  0%|          | 1/262 [00:00<00:59,  4.36it/s]/home/timur.bikbulatov/personal/aa_on_vad/src/datasets/urbansound.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audio = torch.tensor(self.audios[idx], dtype=torch.float32)
100%|██████████| 262/262 [00:13<00:00, 18.73it/s]


precision: 0.8369
recall: 0.9049
f1_score: 0.8696
accuracy: 0.8867
false_positive_rate: 0.1264
true_positive_rate: 0.9049
roc_auc: 0.8892


In [61]:
import importlib
importlib.reload(tl)

<module 'src.models.silero.train' from '/home/timur.bikbulatov/personal/aa_on_vad/src/models/silero/train.py'>

In [9]:
tl.train_noise_producer(
        amodel,
        model,
        train_ds,
        eval_ds,
        epochs=3,
        criterion=AdversarialLoss(alpha=10)
    )

Noise producer model has 25903393 parametes


Epoch 0:   0%|          | 0/66 [00:00<?, ?it/s]

/home/timur.bikbulatov/personal/aa_on_vad/src/datasets/urbansound.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  audio = torch.tensor(self.audios[idx], dtype=torch.float32)
Epoch 0: 100%|██████████| 66/66 [03:28<00:00,  3.16s/it, loss=0.0177]


Epoch 0: Train Loss = 0.0311


Evaluation: 100%|██████████| 9/9 [00:12<00:00,  1.37s/it]


...

Epoch 1: Eval Loss = 0.0038


Epoch 2: 100%|██████████| 66/66 [03:28<00:00,  3.17s/it, loss=-0.0184]


Epoch 2: Train Loss = -0.0012


Evaluation: 100%|██████████| 9/9 [00:12<00:00,  1.38s/it]

Epoch 2: Eval Loss = -0.1288


In [10]:
metrics = validate_silero_vad(
        model,
        eval_ds,
        32,
        amodel,
    )
print_metrics(metrics)

100%|██████████| 9/9 [00:30<00:00,  3.34s/it]


precision: 0.7866
recall: 0.4122
f1_score: 0.5409
accuracy: 0.7079
false_positive_rate: 0.0802
true_positive_rate: 0.4122
roc_auc: 0.6660


In [ ]:
i=9
wav = eval_ds[i]['sample']
mask = eval_ds[i]['mask']
noise = amodel(wav.unsqueeze(0).to(device)).detach().cpu()
plot([wav, mask, noise[0], wav+noise[0]])

In [ ]:
play(wav+amodel(wav.unsqueeze(0).to(device)).detach().cpu())

In [ ]:
play(noise)

In [67]:
wav.shape

torch.Size([96000])

In [14]:
from src.datasets.librispeech import LibriSpeechWrapper
import torchaudio

In [15]:
ds = LibriSpeechWrapper(
    torchaudio.datasets.LIBRISPEECH(root='datasets/torchlibri',
                                     url ='test-clean',
                                     download=True),
                                     sample_rate=16000,
                                    remove_silence_on_edges=False, 
                                    erase_silence=True, 
                                    apply_vad=False)


100%|██████████| 331M/331M [00:36<00:00, 9.44MB/s] 


In [17]:
ds[0]['sample']

{'sample': tensor([ 0.0005, -0.0004, -0.0058,  ...,  0.0026, -0.0023, -0.0124]),
 'sample_rate': 16000,
 'Transcript': 'HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOUR FATTENED SAUCE',
 'speaker_id': 1089,
 'Chapter ID': 134686,
 'Utterance ID': 0}

In [19]:
len(ds)

2620

In [22]:
import numpy as np

def white_noise(n):
    white = np.random.normal(0, 1, n) 
    return white / np.max(white)

def pink_noise(n):
    num_columns = int(np.ceil(np.log2(n)))
    array = np.random.normal(0, 1, (num_columns, int(np.ceil(n / num_columns))))
    pink = array.cumsum(axis=0) / np.sqrt(num_columns)
    pink = pink.flatten()[:n]
    return pink / np.max(pink)

def generate_signal(freq, n, sample_rate=8000):
    """
    Generate a sinusoidal signal at a given frequency.

    Parameters:
        freq (float): Frequency of the sinusoidal signal in Hz.
        n (int): Length of the output array (number of samples).
        sample_rate (int): Sampling rate in Hz (default is 8000 Hz).

    Returns:
        np.ndarray: Array representing the sinusoidal signal.
    """
    t = np.linspace(0, n / sample_rate, n, endpoint=False)
    signal = np.sin(2 * np.pi * freq * t)
    return signal

In [98]:
wav = np.concatenate([
        white_noise(16000) / 10,
        ds[0]['sample'][:16000].numpy(),
        pink_noise(16000*2)/ 10,
        ds[0]['sample'][:-16000*2].numpy(),
])
mask = np.zeros(wav.shape, dtype=np.float32)
mask[16000:16000*2] = 1
mask[4*16000:] = 1

noise = np.zeros(wav.shape, dtype=np.float32)
noise[0::4] = 1
noise[1::4] = 0
noise[2::4] = -1
noise[3::4] = 0
noise = noise * 0.02

x = np.linspace(0, np.pi, 4000)

def sigmoid(x, k):
    return 1 / (1 + np.exp(-k * x))
k = 5
u = sigmoid(np.sin(x), k)
noise[16000:20000] = 10 * noise[16000:20000] + u / 2

In [ ]:
plot([wav, mask, noise], ['Запись', 'Маска', 'Шум'])


In [ ]:
# Define x values
x = np.linspace(0, 10, 1000)

# Sigmoid function
def sigmoid(x, k):
    return 1 / (1 + np.exp(-k * x))

# Create a smoothed square wave using sigmoid
k = 5  # Controls the sharpness of the transition (higher k = sharper)
u = sigmoid(np.sin(x), k)
plot([u])